# Desarrollo de Aplicaciones con Visión Artificial: Semana 1b

En este notebook aprenderás a implementar transformaciones geométricas sobre una imagen.

## Transformación de imágenes

Operaciones comunes en visión artificial son transformar las imágenes. Veamos como rotamos una imagen usando una matriz de transformación e
interpolación bilineal.

In [ ]:
!wget https://github.com/vision-artificial-pucp/invierno-2021/raw/main/cameraman.tif

In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

In [3]:
def interpolar(I, coord):
    if coord[0] < 0 or coord[0] >(I.shape[1]-1) or coord[1] < 0 or coord[1] > (I.shape[0] - 1):
        return 0
    
    minx = math.floor(coord[0])
    maxx = math.ceil(coord[0])
    miny = math.floor(coord[1])
    maxy = math.ceil(coord[1])
    
    dx = coord[0] - minx
    dy = coord[1] - miny
    
    val1 = I[miny,minx]
    val2 = I[miny,maxx]
    val3 = I[maxy,maxx]
    val4 = I[maxy,minx]
    
    val11 = val1*(1-dx) + dx*val2
    val22 = val4*(1-dx) + dx*val3
    
    return val22*(1-dy) + dy*val11

def rotarImagen(I, centro, angulo, dim_salida):
    salida = np.zeros(dim_salida, dtype=np.uint8)
    angle = np.radians(angulo)
    
    T1 = np.array([[1.0, 0.0, -centro[0]],[0.0, 1.0, -centro[1]],[0.0, 0.0, 1.0]])
    T2 = np.array([[np.cos(angle), np.sin(angle), 0.0],[-np.sin(angle), np.cos(angle), 0.0],[0.0, 0.0, 1.0]])
    T3 = np.array([[1.0, 0.0, centro[0]],[0.0, 1.0, centro[1]],[0.0, 0.0, 1.0]])
    
    T = T3 @ T2 @ T1
    T = np.linalg.inv(T)
        
    #Iteramos sobre las posiciones de la salida
    for i in range(salida.shape[0]):
        for j in range(salida.shape[1]):
            coord = np.zeros((3,1), dtype=np.float)
            coord[0] = j
            coord[1] = i
            coord[2] = 1.0
            new_coord = T @ coord
            salida[i,j] = interpolar(I, new_coord)
    
    return salida

In [ ]:
im = cv2.imread('cameraman.tif',0)

rows, cols = im.shape
im2 = rotarImagen(im, centro=(cols/2, rows/2), angulo=45, dim_salida=(rows, cols))

plt.figure(figsize=(10,10))
plt.subplot(1,2,1), plt.imshow(im, cmap='gray')
plt.subplot(1,2,2), plt.imshow(im2, cmap='gray')
plt.show()

Podemos aplicar implementaciones más eficientes como las de OpenCV. Por ejemplo, veamos cómo escalar y rotar una imagen.

In [ ]:
img = cv2.imread('cameraman.tif')
print(img.shape)

result = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
print(result.shape)

Y la operación de rotación.

In [ ]:
rows, cols, _ = img.shape

M = cv2.getRotationMatrix2D((cols/2, rows/2), 45, 1)
# Sirve para rotacion, escala, traslacion y shear
result = cv2.warpAffine(img, M, (cols, rows))

plt.figure(figsize=(10,10))
plt.subplot(1, 2, 1), plt.imshow(img)
plt.subplot(1, 2, 2), plt.imshow(result)
plt.show()

Una transformación más general es la transformación en perspectiva que permite corregir objetos con deformaciones más complejas.

In [ ]:
!wget https://github.com/vision-artificial-pucp/invierno-2021/raw/main/documento.jpg

In [ ]:
#Leemos la imagen a corregir
img = cv2.imread('documento.jpg')
img = img[:,:,::-1]

plt.figure(figsize=(10,10))
plt.imshow(img)
plt.show()

In [ ]:
# Definimos los puntos de las esquinas del papel
# Dibujamos los puntos y el poligono en la imagen
puntos = np.float32([[731, 1158],[2398, 1138],[3047, 3778], [132, 3839]])

imgDibujo = img.copy()

for i in range(0, 4):
  cv2.circle(imgDibujo, tuple(puntos[i]), 30, (255,0,0), -1)

pts = puntos.reshape((-1,1,2))
cv2.polylines(imgDibujo, [pts.astype(np.int32)], True, (0,255,0), thickness=10)

plt.figure(figsize=(10,10))
plt.imshow(imgDibujo)
plt.show()

In [ ]:
width = 500
height = 700

# Hacemos la transformacion
target = np.float32([[0,0],[width,0],[width,height],[0,height]])

matrix = cv2.getPerspectiveTransform(puntos, target)
result = cv2.warpPerspective(img, matrix, (width,height))

plt.figure(figsize=(10,10))
plt.imshow(result)
plt.show()

## Ejercicio 2

* Crear una función que realice un flip horizontal sobre una imagen.
* Crear una función que realice un flip vertical sobre una imagen.

In [ ]:
img = cv2.imread('cameraman.tif')

#flip_vertical = ...
#flip_horizontal = ...

plt.figure(figsize=(10,10))
plt.subplot(1,3,1), plt.imshow(img)
plt.subplot(1,3,2), plt.imshow(flip_vertical)
plt.subplot(1,3,3), plt.imshow(flip_horizontal)
plt.show()